In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import statsmodels.api as sm
import time
import pickle

In [288]:
data = pd.read_csv('loan.csv')

In [289]:
for col in data.columns:
    print(col)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
b

## Preprocessing

In [290]:
bad_loan = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", 
            "In Grace Period", "Late (16-30 days)", "Late (31-120 days)"]

def loan_condition(status):
    if status in bad_loan:
        return 1
    else:
        return 0
    
    
y = data['loan_status'].apply(loan_condition)
y.name = 'bad_loan'

In [291]:
def term_str_to_int(term_str):
    return [int(s) for s in term_str.split() if s.isdigit()][0]
data['term'] = data['term'].apply(term_str_to_int)

In [292]:
def grade_str_to_int(grade_str):
    return ord(grade_str.lower()) - 97
data['grade'] = data['grade'].apply(grade_str_to_int)

In [293]:
def subgrade_str_to_int(subgrade_str):
    return grade_str_to_int(subgrade_str[0]) * 5 + (int(subgrade_str[1])-1)
data['sub_grade'] = data['sub_grade'].apply(subgrade_str_to_int)

In [294]:
def emplength_str_to_int(emplength_str):
    if pd.isnull(emplength_str):
        return np.nan
    else:
        return int(re.sub('[^0-9]+', '', str(emplength_str)))
data['emp_length'] = data['emp_length'].apply(emplength_str_to_int)

In [295]:
def pymntplan_str_to_int(pymntplan_str):
    if pymntplan_str == 'y':
        return 1
    else:
        return 0
data['pymnt_plan'] = data['pymnt_plan'].apply(pymntplan_str_to_int)

In [296]:
data['issue_d'] = pd.to_datetime(data['issue_d'], format='%b-%Y')
data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'], format='%b-%Y')
data['sec_app_earliest_cr_line'] = pd.to_datetime(data['sec_app_earliest_cr_line'], 
                                                  format='%b-%Y')
def month_diff(a, b):
    return 12 * (a.dt.year - b.dt.year) + (a.dt.month - b.dt.month)

data['earliest_cr_line'] = month_diff(data['issue_d'], data['earliest_cr_line'])
data['sec_app_earliest_cr_line'] = month_diff(data['issue_d'], 
                                              data['sec_app_earliest_cr_line'])

In [297]:
def applicationtype_str_to_int(applicationtype_str):
    if applicationtype_str == 'Joint App':
        return 1
    else:
        return 0
data['application_type_joint'] = data['application_type'].apply(applicationtype_str_to_int)

In [298]:
def disbursement_str_to_int(disbursement_str):
    if disbursement_str == 'Cash':
        return 1
    else:
        return 0
data['disbursement_method_cash'] = data['disbursement_method'].apply(disbursement_str_to_int)

In [299]:
to_drop = ['id', 'member_id', 'emp_title', 'issue_d', 'loan_status', 'url', 'desc', 
           'title', 'zip_code', 'initial_list_status', 'last_pymnt_d', 'next_pymnt_d',
           'last_credit_pull_d', 'policy_code', 'application_type', 'hardship_type',
           'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 
           'hardship_length', 'hardship_loan_status', 'debt_settlement_flag',
           'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
           'settlement_amount', 'settlement_percentage', 'settlement_term',
           'deferral_term', 'hardship_flag', 'hardship_type', 'hardship_reason',
           'hardship_status', 'hardship_amount', 'hardship_start_date',
           'hardship_end_date', 'payment_plan_start_date', 'hardship_length',
           'hardship_dpd', 'hardship_loan_status', 
           'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
           'hardship_last_payment_amount', 'disbursement_method', 'recoveries', 
           'collection_recovery_fee', 'delinq_amnt']
data.drop(columns=to_drop, inplace=True)

In [300]:
to_dummies = ['home_ownership', 'verification_status', 'purpose', 'addr_state', 
              'verification_status_joint']
data = pd.get_dummies(data, prefix=to_dummies, columns=to_dummies)

In [301]:
iffy = ['mths_since_last_delinq', 'mths_since_last_record', 'mths_since_last_major_derog',
        'mths_since_rcnt_il', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
        'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mths_since_recent_bc',
        'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 
        'mths_since_recent_revol_delinq', 'sec_app_mths_since_last_major_derog']
data.drop(columns=iffy, inplace=True)

In [333]:
fill_na_100 = ['dti', 'pct_tl_nvr_dlq']
data[fill_na_100] = data[fill_na_100].fillna(100)

data.fillna(0, inplace=True)

In [336]:
data.to_pickle('data.pkl.gzip')

In [337]:
y.to_pickle('y.pkl.gzip')

## Prediction

In [3]:
data = pd.read_pickle('data.pkl.gzip')
y = pd.read_pickle('y.pkl.gzip')

In [4]:
for col in data.columns:
    print(col)

loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_length
annual_inc
pymnt_plan
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
open_acc
pub_rec
revol_bal
revol_util
total_acc
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
last_pymnt_amnt
collections_12_mths_ex_med
annual_inc_joint
dti_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
chargeoff_within_12_mths
mort_acc
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_120dpd_2m
num_tl_30dpd
num_tl_90g_dpd_24m
num_tl_op_past_12m
pct_tl_nvr_dlq
percent_bc_gt_75
pub_rec_bankruptcies
tax_liens
tot_hi_cred_lim
total_bal_ex_mort
total_bc_limit
total_

In [7]:
data = data.loc[:,~data.columns.str.startswith('addr_state')]

In [27]:
pos_samples = y[y == 1].sample(frac=0.1, axis=0).index
neg_samples = y[y == 0].sample(len(pos_samples), axis=0).index
samples = pos_samples.union(neg_samples)
len(samples)

59406

In [28]:
current_cols = []
possible_cols = list(data.columns)

for i in range(len(data.columns)):
    aics = []
    old_pvalues = []
    added_pvalues = []
    
    print('')
    print('----------------------------------------')

    for col in possible_cols:
        try:
            clf = sm.Logit(
                y[samples], 
                data[current_cols + [col]].loc[samples]
            ).fit(full_output=False, disp=False)
            aics.append(clf.aic)
            old_pvalues.append(clf.pvalues[:-1])
            added_pvalues.append(clf.pvalues[-1])
        except:
            aics.append(np.inf)
            old_pvalues.append([])
            added_pvalues.append(1)
            
    
    lowest_aic = 1e10
    for pvalue, aic in zip(added_pvalues, aics):
        if pvalue < 0.05 and aic < lowest_aic:
            lowest_aic = aic
            
    if lowest_aic == 1e10:
        print(added_pvalues)
        break
        
    best_col = aics.index(lowest_aic)
    
    print(f'--Best col: {possible_cols[best_col]} p-value={added_pvalues[best_col]}, aic={aics[best_col]}--')

    current_cols.append(possible_cols.pop(best_col))
    
    print(f'---- Current columns ----')
    for i, pvalue in enumerate(old_pvalues[best_col]):
        print(f'{current_cols[i]}: pvalue={pvalue}')
        if pvalue > 0.2:
            print(f'--Remove col: {current_cols[i]}, p-value={pvalue}--')
            possible_cols.append(current_cols.pop(i))
    
print('DONE')
print(current_cols)


----------------------------------------


/Users/jurrew27/anaconda3/envs/quant/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '


--Best col: last_pymnt_amnt p-value=0.0, aic=72664.54681497569--
---- Current columns ----

----------------------------------------
--Best col: grade p-value=0.0, aic=61808.77904855338--
---- Current columns ----
last_pymnt_amnt: pvalue=0.0

----------------------------------------
--Best col: out_prncp p-value=0.0, aic=53472.13062919433--
---- Current columns ----
last_pymnt_amnt: pvalue=0.0
grade: pvalue=0.0

----------------------------------------
--Best col: funded_amnt p-value=0.0, aic=49929.39417438021--
---- Current columns ----
last_pymnt_amnt: pvalue=0.0
grade: pvalue=0.0
out_prncp: pvalue=0.0

----------------------------------------


/Users/jurrew27/anaconda3/envs/quant/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '


--Best col: total_pymnt p-value=0.0, aic=35311.81398712956--
---- Current columns ----
last_pymnt_amnt: pvalue=0.0
grade: pvalue=0.0
out_prncp: pvalue=0.0
funded_amnt: pvalue=0.0

----------------------------------------


...

--Best col: sec_app_open_act_il p-value=0.0403358984600653, aic=17546.972033540005--
---- Current columns ----
last_pymnt_amnt: pvalue=2.8509691090224542e-71
out_prncp: pvalue=1.8786641916980093e-161
funded_amnt: pvalue=2.6564879445357892e-160
total_pymnt: pvalue=1.4731815307734071e-164
total_rec_int: pvalue=1.8497218519835593e-165
pct_tl_nvr_dlq: pvalue=0.06375021178745488
total_rec_late_fee: pvalue=2.7827398354500693e-186
open_rv_24m: pvalue=8.809674024164458e-07
num_accts_ever_120_pd: pvalue=0.0003262372892364197
il_util: pvalue=0.043224856277209595
emp_length: pvalue=7.646886501639747e-05
sub_grade: pvalue=3.423727974740332e-33
int_rate: pvalue=2.0702213126512945e-19
disbursement_method_cash: pvalue=4.240340662917417e-09
total_il_high_credit_limit: pvalue=0.001759017544885583
inq_last_12m: pvalue=0.0022390554524427124
installment: pvalue=4.7671130979949336e-08
all_util: pvalue=8.063556980669215e-06
sec_app_inq_last_6mths: pvalue=0.010858411857905429
verification_status_Not Verified

/Users/jurrew27/anaconda3/envs/quant/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '


--Best col: sec_app_open_acc p-value=0.0028900309621036594, aic=17539.58246400952--
---- Current columns ----
last_pymnt_amnt: pvalue=3.9051696509580945e-71
out_prncp: pvalue=1.8051674561899077e-161
funded_amnt: pvalue=2.268017844105828e-160
total_pymnt: pvalue=1.2551363066469454e-164
total_rec_int: pvalue=1.759612036409452e-165
pct_tl_nvr_dlq: pvalue=0.06406942019960608
total_rec_late_fee: pvalue=1.0883199965128994e-185
open_rv_24m: pvalue=8.324418110841647e-07
num_accts_ever_120_pd: pvalue=0.0003243649565588804
il_util: pvalue=0.04521529571603702
emp_length: pvalue=4.366144521299691e-05
sub_grade: pvalue=3.801971826963772e-33
int_rate: pvalue=2.2854270348447343e-19
disbursement_method_cash: pvalue=2.208455664561805e-09
total_il_high_credit_limit: pvalue=0.001519311474464036
inq_last_12m: pvalue=0.0024423412942323817
installment: pvalue=4.515605308283104e-08
all_util: pvalue=7.364472543358077e-06
sec_app_inq_last_6mths: pvalue=0.0004566604852557221
verification_status_Not Verified: pv

/Users/jurrew27/anaconda3/envs/quant/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '


[0.6685506111195904, 0.33512773861964607, 0.9942347484630961, 0.5699864325692647, 0.9548680743229345, 0.604213269435854, 0.7029563882219021, 0.48061511417380154, 0.41541281123819973, 0.409536272221782, 0.2530203303208697, 0.24723237638399265, nan, 0.3848180838266666, 0.09074365976029328, 0.2186169918685753, 0.28139236212799346, 0.5962638382002234, 0.6017667313365187, 0.10541499019576393, 0.6563071730827938, 0.4570288994444738, 0.5551102472037188, 0.34265455543031853, 0.9440238335536306, 0.9321644434598609, 0.5378236686286428, 0.7104288437950121, 0.6399410277407745, 0.07748197583580009, 0.9281474729793213, 0.9927264819245584, 0.6118782478881091, 0.9756499799205155, 0.4435948431886536, 0.7529209311319756, 0.995340915870954, 0.9947305152426392, 0.25137162198657315, 0.06648577753311397, 0.3995645262004234, 0.12332705244328075, 0.525040908352294, 0.33363812900047074, 0.5084381281466209, 0.9105033698400882, 0.5350454659495576, 0.45374531432245035, 0.8750128611577029, 0.9621589885974682, 0.83

In [29]:
best_params = ['last_pymnt_amnt', 'out_prncp', 'funded_amnt', 'total_pymnt', 'total_rec_int', 'pct_tl_nvr_dlq', 'total_rec_late_fee', 'open_rv_24m', 'num_accts_ever_120_pd', 'il_util', 'emp_length', 'sub_grade', 'int_rate', 'disbursement_method_cash', 'total_il_high_credit_limit', 'inq_last_12m', 'installment', 'all_util', 'sec_app_inq_last_6mths', 'verification_status_Not Verified', 'revol_util', 'verification_status_Source Verified', 'verification_status_Verified', 'num_bc_tl', 'total_bc_limit', 'num_tl_90g_dpd_24m', 'open_acc', 'num_actv_rev_tl', 'term', 'purpose_small_business', 'home_ownership_MORTGAGE', 'open_il_24m', 'annual_inc', 'sec_app_open_act_il', 'sec_app_open_acc']

In [30]:
clf = sm.Logit(y[samples], data[best_params].loc[samples]).fit()

Optimization terminated successfully.
         Current function value: 0.147036
         Iterations 15


In [35]:
y[samples].to_pickle('y_samples.pkl')

In [37]:
with open('logit_result.pkl', 'wb') as f:
    pickle.dump(clf, f)